In [ ]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
import os
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [ ]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

# logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 404, 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2_Fees' in pwd:
    prepend = ''
else:
    prepend = 'L2_Fees/'

In [ ]:
trailing_num_days = 90

start_date = date.today()-timedelta(days=trailing_num_days +1)

# start_date = datetime.strptime('2022-07-13', '%Y-%m-%d').date()

In [ ]:
statuses = {x for x in range(100, 600)}
statuses.remove(200)
statuses.remove(429)

In [ ]:
async def get_cryptostats_api(api_core_string,dt_string ):
        dt_string = dt_string.strftime("%Y-%m-%d")
        # print(dt_string)
        i = 0
        res = pd.DataFrame()
        # #try 5 times
        # while i < 5:
        #         try: 
        async with aiohttp.ClientSession() as session:
                retry_client = RetryClient(session)
                async with retry_client.get(api_core_string + dt_string, retry_options=ExponentialRetry(attempts=3), raise_for_status=statuses) as response:
                        try:
                                prot_req = await response.json()
                        except Exception as e:
                                raise Exception("Could not convert json")
                await retry_client.close()
                
                # prot_req = await retry_client.get(api_core_string + dt_string, headers=header)
                # prot_req = await prot_req.json()
        res = pd.json_normalize(prot_req['data']).reset_index()
        res = res[res['metadata.blockchain'] == 'Optimism']
        res['date'] = dt_string
                # except:
                #         i = i+1
                #         time.sleep(1)
                #         continue
                # break
        return res

# async def gather_with_concurrency(n, *tasks):
#     semaphore = asyncio.Semaphore(n)
#     async def sem_task(task):
#         async with semaphore:
#             return await task
        

In [ ]:
# ALL Relevant API Strings
# https://cryptostats.community/discover/fees
# This is dependent on crypto stats, eventually maybe we mod this to pull from TheGraph, but this is simplest por ahora
# https://docs.cryptostats.community/tutorial/how-to-query-with-rest-api/making-get-requests
        #How to make a single query
        #To make a single query on a collection. Send a GET request to
        #https://api.cryptostats.community/api/v1/<collection-id>/<query>/<arg>
        
        # How to make multiple queries
        # To make multiple queries on a collection, send a GET request to
        # https://api.cryptostats.community/api/v1/<collection-id>/<query-1>,<query-2>/<query-1-arg>,<query-2-arg>

api_core = 'https://api.cryptostats.community/api/v1/fees/oneDayTotalFees/' #Can only pull everything all at once. Odd.
date_rng = pd.date_range(start=start_date, end=date.today()-timedelta(days=1))
date_rng_str = ','.join(x.strftime("%Y-%m-%d") for x in date_rng)
# print(date_rng[0].strftime("%Y-%m-%d"))

In [ ]:
def get_range(date_range):
        data_dfs = []
        fee_df = []
        # for dt in date_range:
        #         await asyncio.gather()
        #         data_dfs.append(res_df)
        #         # res.columns
        # try:
        #         loop.close()
        # except:
        #         #nothing
        loop = asyncio.get_event_loop()
        tasks = [get_cryptostats_api(api_core,dt) for dt in date_range]
        # print(tasks)
        data_dfs = loop.run_until_complete(asyncio.gather(*tasks, return_exceptions=True))
        # print(date_range)
        # loop.close()
        print(data_dfs)
        fee_df = pd.concat(data_dfs)
        return fee_df

In [ ]:


fdf = get_range(date_rng)
# r.get('https://api.cryptostats.community/api/v1/fees/oneDayTotalFees/2022-05-21')
# print(r)

In [ ]:
display(fdf)